In [1]:
!git clone https://github.com/mykola-mmm/airbus_kaggel.git

Cloning into 'airbus_kaggel'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 190 (delta 19), reused 10 (delta 4), pack-reused 155
Receiving objects: 100% (190/190), 99.99 MiB | 34.03 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [2]:
import os
os.chdir('./airbus_kaggel')


In [3]:
!git checkout v2

Branch 'v2' set up to track remote branch 'v2' from 'origin'.
Switched to a new branch 'v2'


In [4]:
!python --version

Python 3.10.13


In [5]:
!python ./train-model.py --csv_file /kaggle/input/airbus-kaggle-preprocessed/df.csv --dataset_path /kaggle/input/airbus-ship-detection/train_v2 --max_number_of_samples 1500 --epochs 300 --batch_size 16 --loss_function dice_bce_loss --patch_size 768

2024-07-26 20:10:31.911447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 20:10:31.911548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 20:10:32.057991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Number of devices: 2
The size of the training set: 945
The size of the validation set: 237
Steps/Epoch: 59
<class 'function'>
Model: "functional"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━

In [6]:
!ls -la


total 49596
drwxr-xr-x 7 root root     4096 Jul 26 20:12 .
drwxr-xr-x 3 root root     4096 Jul 26 20:10 ..
drwxr-xr-x 8 root root     4096 Jul 26 20:10 .git
-rw-r--r-- 1 root root       43 Jul 26 20:10 .gitignore
-rw-r--r-- 1 root root     5040 Jul 26 20:10 README.md
-rw-r--r-- 1 root root 50719787 Jul 26 20:10 df.csv
drwxr-xr-x 4 root root     4096 Jul 26 20:21 logs
drwxr-xr-x 2 root root     4096 Jul 26 20:24 models
drwxr-xr-x 2 root root     4096 Jul 26 20:10 notebooks
-rw-r--r-- 1 root root      128 Jul 26 20:10 requirements.txt
-rw-r--r-- 1 root root     2878 Jul 26 20:10 test-model.py
-rw-r--r-- 1 root root    13222 Jul 26 20:10 train-model.py
drwxr-xr-x 3 root root     4096 Jul 26 20:10 utils
